In [33]:
%pip install git+https://github.com/yfukai/tracksdata@ensure_layer_data_protocol

  Cloning https://github.com/yfukai/tracksdata (to revision ensure_layer_data_protocol) to /private/var/folders/81/kh0zpmbd00g39vwz69lkk4mc0000gn/T/pip-req-build-7p184wbg
  Running command git clone --filter=blob:none --quiet https://github.com/yfukai/tracksdata /private/var/folders/81/kh0zpmbd00g39vwz69lkk4mc0000gn/T/pip-req-build-7p184wbg
  Running command git checkout -b ensure_layer_data_protocol --track origin/ensure_layer_data_protocol
  Switched to a new branch 'ensure_layer_data_protocol'
  Branch 'ensure_layer_data_protocol' set up to track remote branch 'ensure_layer_data_protocol' from 'origin'.
  Resolved https://github.com/yfukai/tracksdata to commit 2970dd421a4e59d7404f0961014994c636ca2541
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for tracksdata: filename=tracksdata-0.1.dev615+g2970dd421-py3-none-any.whl size=191359 sha256=ef99b4675040eb41c7cadd9495743b42c2154a591d

In [14]:
image_path = "/Volumes/work/fukai/241203-cell-picker/aligned_image.zarr"
graph_path = "/Users/fukai/Downloads/aligned_labels_tracked3.db"

In [8]:
import tracksdata as td
import os
os.environ["NAPARI_ASYNC"] = "1"
import zarr
from dask import array as da

In [15]:
image = da.from_zarr(zarr.open(image_path, mode="r"))

In [9]:
graph = td.graph.SQLGraph(drivername="sqlite", database=graph_path)

In [16]:
gav = td.array.GraphArrayView(graph, shape=tuple([image.shape[0], *image.shape[2:]]), attr_key="label")

In [17]:
from napari import Viewer
from napari_travali2._stateful_widget import StateMachineWidget
from napari_travali2._logging import logger
from napari_travali2.actionable_tracks import actionable_tracks as at

In [27]:

import numpy as np

logger.setLevel("DEBUG")
logger.info("Starting napari-travali2")

viewer = Viewer()
#widget = StateMachineWidget(viewer, ta, image, 
#                            verified_track_ids=ta.attrs.get("verified_track_ids", []),
#                            candidate_track_ids=ta.attrs.get("candidate_track_ids", []),
#                            crop_size=2048)
#viewer.window.add_dock_widget(widget, area="right")
#viewer.dims.set_current_step(0,0)

In [39]:
layer = viewer.add_labels([gav, gav[:,::2,::2]], name="tracks")

In [33]:
from os import path
LOGGING_PATH = ".napari-travali/log.txt"
import logging 
import numpy as np
import polars as pl

SELECTED_COLOR = (1, 0, 0, 1)
DAUGHTER_COLORS = [
    (0, 1, 0, 1),
    (0, 0, 1, 1)
]
logging_path = path.join(path.expanduser("~"), LOGGING_PATH)
os.makedirs(path.dirname(logging_path), exist_ok=True)

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)  # Set the logger level to DEBUG
file_handler = logging.FileHandler(logging_path)
file_handler.setLevel(logging.DEBUG)  # Set the handler level to DEBUG
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)
logger.propagate = False  # Prevent propagation to the root logger
logger.info("Track clicked")


def find_track_successors(graph, bbox_spatial_filter, data_coordinates) -> tuple[int, pl.DataFrame]:
    logger.debug(f"data_coordinates: {data_coordinates}")
    nodes_df = bbox_spatial_filter[tuple([
        slice(c,c) for c in data_coordinates
    ])].node_attrs(attr_keys=["node_id", "label"])
    logger.debug("node_df built.")
    if nodes_df.is_empty():
        logger.info("No nodes in the bbox.")
        return -1, pl.DataFrame()
    track_id = nodes_df["label"].to_list()[0]
    # TODO make the following faster
    sorted_node_ids = graph.filter(td.NodeAttr("label") == track_id).node_attrs(attr_keys=["node_id","t"]).sort("t")["node_id"]
    logger.debug("sorted_node_ids built.")
    last_node_id = sorted_node_ids.last()
    #successors_df = graph.successors(last_node_id, return_attrs=True, attr_keys=["track_id"])
    successors_df = graph.successors(last_node_id, attr_keys=["label"])
    logger.debug("successors_df built.")
    return track_id, successors_df

def track_clicked(viewer, event):
    logger.info(event.modifiers)
    logger.info("Track clicked")
    yield  # important to avoid a potential bug when selecting the daughter
    logger.info("button released")
    data_coordinates = layer.world_to_data(event.position)
    logger.debug(f"world coordinates: {event.position}")
    logger.debug(f"data coordinates: {data_coordinates}")
    track_id, successors_df = find_track_successors(graph, gav._spatial_filter, data_coordinates)
    if track_id == -1:
        logger.info("No track found.")
        return
    if len(successors_df) == 0:
        logger.info("No successors found.")
        successor_track_ids = []
    else:
        successor_track_ids = successors_df["track_id"].to_list()
    # get the row of nodes_df (polars dataframe) with the selected track_id and largest t
    #first_node_id = sorted_node_ids.first()
    logger.info(f"selected track_id: {track_id}, "
                f"successor track_ids: {successor_track_ids}.")
    layer.colormap = {0:(0,0,0,0), 
                    track_id:SELECTED_COLOR,
                **{d:c for d,c in zip(successor_track_ids,DAUGHTER_COLORS)},
                    None:(0,0,0,0)}
    logger.info("colormap updated.")
#    selected_graph = graph.filter(td.NodeAttr("track_id").is_in([track_id, *successor_track_ids])).subgraph()
#    graph_view = td.array.GraphArrayView(selected_graph, labels.shape, attr_key="track_id")
#    selected_layer.data = graph_view
    
    
viewer.mouse_drag_callbacks.clear()
viewer.mouse_drag_callbacks.append(track_clicked)

In [43]:
from napari.utils.colormaps import label_colormap
layer.colormap = label_colormap(
            49, 0, background_value=0
    )

In [40]:
layer.colormap = {0:(0,0,0,0), None:(0,0,0,0)}

In [44]:
track_id = 181254
layer.colormap = {0:(0,0,0,0), 
                track_id:SELECTED_COLOR,
                None:(0,0,0,0)}

In [35]:
layer.refresh()

In [12]:
tracks = at.ActionableTracks(graph, tracklet_id_attr_name="label")

In [7]:
viewer.add_image([image[:,0],image[:,0,::2,::2]], name="image")

<Image layer 'image' at 0x325541100>

In [29]:
viewer.add_labels([gav, gav[:,::2,::2]], name="tracks")

<Labels layer 'tracks' at 0x315e18b60>

In [10]:
gav2 = gav[:,5000:6000,5000:6000]
viewer.add_labels([gav2, gav2[:,::2,::2]], name="tracks")

<Labels layer 'tracks' at 0x397be0a70>

In [28]:
%time np.asarray(gav[0])

CPU times: user 90 ms, sys: 210 ms, total: 300 ms
Wall time: 496 ms


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [31]:
%time np.asarray(gav[1000,5000:6000,5000:6000])

CPU times: user 23.4 ms, sys: 24.1 ms, total: 47.5 ms
Wall time: 52.2 ms


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [13]:
%time np.asarray(gav[1904])

CPU times: user 8 μs, sys: 22 μs, total: 30 μs
Wall time: 31 μs


NameError: name 'gav' is not defined